## Execute the steps manually in the notebook ##

In [ ]:
import requests
import time
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

### Ensure that the API URL is the correct one bellow

In [ ]:
API_BASE_URL = "https://flask-api-1069651367433.us-east4.run.app"
headers = {
    "Content-Type": "application/json",
}

### Function to Retreive API Key

In [ ]:
def retrieve_api_key():
    username = input("Enter your username: ")
    password = input("Enter your password: ")

    auth_data = {"username": username, "password": password}
    try:
        response = requests.post(f"{API_BASE_URL}/get_api_key", json=auth_data, headers=headers, timeout=10)
        if response.status_code == 200:
            api_key = response.json().get("api_key")
            logging.info(f"API key retrieved successfully: {api_key}")
            headers["x-api-key"] = api_key
            return api_key
        else:
            logging.error(f"Error retrieving API key: {response.status_code} - {response.text}")
            return None
    except requests.exceptions.RequestException as e:
        logging.error(f"Network error while retrieving API key: {e}")
        return None


### Function to Obtain prompt and data

In [ ]:
def get_job_data_from_user():
    client_id = input("Enter client ID: ")
    project_id = input("Enter project ID: ")
    dataset_id = input("Enter dataset ID: ")
    table_id = input("Enter table ID: ")
    table_key = input("Enter table key (as JSON object): ") or "{}"
    row_count = int(input("Enter row count: "))
    request_column = int(input("Enter request column index: "))
    response_column = int(input("Enter response column index: "))
    llm_model = input("Enter LLM model (e.g., gpt-3.5): ")
    prompt_prefix = input("Enter prompt prefix: ")
    prompt_postfix = input("Enter prompt postfix: ")

    return {
        "client_id": client_id,
        "project_id": project_id,
        "dataset_id": dataset_id,
        "table_id": table_id,
        "table_key": table_key,
        "row_count": row_count,
        "request_column": request_column,
        "response_column": response_column,
        "llm_model": llm_model,
        "prompt_prefix": prompt_prefix,
        "prompt_postfix": prompt_postfix,
    }


### Function to Submit Job

In [ ]:
def submit_job():
    job_data = get_job_data_from_user()

    try:
        response = requests.post(f"{API_BASE_URL}/submit_job", json=job_data, headers=headers, timeout=10)
        if response.status_code == 200:
            logging.info("Job submitted successfully.")
            return response.json()['job_id']
        else:
            logging.error(f"Error submitting job: {response.status_code} - {response.text}")
            return None
    except requests.exceptions.RequestException as e:
        logging.error(f"Network error occurred: {e}")
        return None


### Function to Check Job Status

In [ ]:
def check_job_status(job_id):
    try:
        response = requests.get(f"{API_BASE_URL}/job_status/{job_id}", headers=headers, timeout=10)
        if response.status_code == 200:
            return response.json()
        elif response.status_code == 404:
            logging.warning(f"Job with ID {job_id} not found.")
        else:
            logging.error(f"Failed to retrieve job status: {response.status_code} - {response.text}")
    except requests.exceptions.RequestException as e:
        logging.error(f"Network error while checking job status: {e}")
    return None


### Script to initialize the Job

In [ ]:
# Step 1: Retrieve API Key
api_key = retrieve_api_key()

# Step 2: Submit a Job
if api_key:
    job_id = submit_job()
    if job_id:
        logging.info(f"Job ID: {job_id}")

        # Step 3: Poll Job Status
        backoff_time = 30
        max_backoff_time = 3600
        max_attempts = 10

        attempts = 0
        while attempts < max_attempts:
            status = check_job_status(job_id)
            if status:
                logging.info(f"Job status: {status['status']}")
                if status['status'] == "completed":
                    logging.info("Job completed successfully!")
                    break
                elif status['status'] == "processing":
                    logging.info("Job is still processing. Checking again in a few seconds...")
            else:
                logging.warning("Failed to retrieve job status. Retrying...")
            time.sleep(backoff_time)
            backoff_time = min(backoff_time * 2, max_backoff_time)
            attempts += 1

        if attempts == max_attempts:
            logging.error("Maximum number of attempts reached. Job status check terminated.")
